In [1]:
import pandas as pd
import numpy as np
import requests
import os
import json
import time

In [2]:
coastal_api = os.getenv('coastal_api')
r = requests.get('http://httpbin.org/user-agent')
useragent = json.loads(r.text)['user-agent']

In [3]:
# https://coastal.er.usgs.gov/hurricanes/research/twlviewer/api/regions/381624075071901/forecasts

In [4]:
headers = {'User-Agent': useragent,
           'From': 'zrc3hc@virginia.edu',
          'token': coastal_api}

In [5]:
sites_df = pd.read_csv('sites_data.csv')
sites_df = sites_df.drop('Unnamed: 0', axis= 1)
sites_df

,id,siteLatitude,siteLongitude,location,city,stateOrTerritory,siteFeatureType,toeHeight,crestHeight,dateElevationMeasured,...,forecastLongitude,distanceBetweenSiteAndForecast,beachSlope,maxPredictedImpact,maxPredictedImpactCode,predictedDurationOfCollision,predictedDurationOfOverwash,predictedDurationOfInundation,region_id,forecast_id
0,1450,41.3285,-70.819,Gay Head Town Beach,Chilmark,MA,Dune,2.99,7.00,2013-11-01,...,-70.812,896,0.1560,NaN,0,0,0,0,1,72261
1,1451,41.3224,-70.812,Long Beach,Chilmark,MA,Cliff,3.10,7.31,2013-11-01,...,-70.812,0,0.1560,NaN,0,0,0,0,1,72261
2,1452,41.3195,-70.807,Long Beach,Chilmark,MA,Dune,3.49,7.14,2013-11-01,...,-70.807,0,0.0950,NaN,0,0,0,0,1,72261
3,1453,41.3114,-70.794,Squibnocket Marshes,Chilmark,MA,Dune,3.34,7.42,2013-11-01,...,-70.794,0,0.0874,NaN,0,0,0,0,1,72261
4,1454,41.3021,-70.778,Squibnocket Point,Chilmark,MA,Dune,2.70,7.29,2013-11-01,...,-70.778,0,0.1102,NaN,0,0,0,0,1,72261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6570,1470,41.3489,-70.630,Long Cove,West Tisbury,MA,Dune,2.07,3.23,2013-11-01,...,-70.630,0,0.1114,NaN,0,0,0,0,1,72249
6571,1472,41.3499,-70.606,Ripley Cove,West Tisbury,MA,Dune,2.47,4.26,2013-11-01,...,-70.606,0,0.1211,NaN,0,0,0,0,1,72249
6572,1473,41.3504,-70.595,Long Point Wildlife Refuge,West Tisbury,MA,Dune,2.37,4.04,2013-11-01,...,-70.595,0,0.1168,NaN,0,0,0,0,1,72249
6573,1474,41.3504,-70.589,Long Point Wildlife Refuge,West Tisbury,MA,Dune,2.30,3.98,2013-11-01,...,-70.589,0,0.1134,NaN,0,0,0,0,1,72249


In [6]:
# Assuming df is your DataFrame
id_pairs = [(row['id'], row['region_id'], row['forecast_id']) for index, row in sites_df.iterrows()]

In [ ]:
headers = {'User-Agent': useragent,
           'From': 'zrc3hc@virginia.edu',
          'token': coastal_api}

pause_duration = 1  
request_counter = 0
requests_per_batch = 1

water_levels_info= []
 
for site_id, region_id, forecast_id in id_pairs:

    root = f'https://coastal.er.usgs.gov/hurricanes/research/twlviewer/api/regions/{region_id}/forecasts/{forecast_id}/sites/{site_id}/waterlevels'

    r = requests.get(root, headers=headers)

    # Check if the request was successful
    if r.status_code == 200:
        water_data = r.json()
        for site in water_data:
            site['region_id'] = region_id
            site['forecast_id'] = forecast_id
            site['site_id'] = site_id
        water_levels_info.extend(water_data)
    else: 
        print(f"Failed to retrieve data for region ID {region_id}, forecast ID {forecast_id}, site ID {site_id}")

    # Increment the request counter
    request_counter += 1

    # Check if it's time to pause and reset the counter
    if request_counter >= requests_per_batch:
        print(f"Pausing for {pause_duration} seconds after {requests_per_batch} requests.")
        time.sleep(pause_duration)
        request_counter = 0